# 01 - LangChain 基礎

## 📚 學習目標

- ✅ 理解 LangChain 核心概念與架構
- ✅ 掌握 PromptTemplate 的使用方法
- ✅ 學習 LLM 與 ChatModel 的差異
- ✅ 建立第一個 Chain 應用
- ✅ 實作國泰人壽保單查詢助理

## ⏰ 預計時間: 45 分鐘

## 📦 環境設定

### 安裝必要套件

In [ ]:
# 安裝 LangChain 核心套件
!pip install langchain==0.1.0
!pip install langchain-openai==0.0.5
!pip install langchain-core==0.1.10
!pip install python-dotenv==1.0.0

In [ ]:
# 載入環境變數
from dotenv import load_dotenv
import os

load_dotenv()
print("✅ 環境設定完成")
print(f"OpenAI API Key: {'已設定' if os.getenv('OPENAI_API_KEY') else '未設定'}")

## Part 1: LangChain 核心概念

### 1.1 LangChain 架構概覽

```
┌─────────────────────────────────────────┐
│         LangChain 架構層次              │
├─────────────────────────────────────────┤
│  Application Layer (你的應用)           │
├─────────────────────────────────────────┤
│  Chains (串接多個組件)                  │
├─────────────────────────────────────────┤
│  Components:                            │
│  - Prompts (提示詞模板)                 │
│  - Models (LLM/ChatModel)              │
│  - Output Parsers (輸出解析)           │
│  - Memory (記憶)                        │
│  - Tools (工具)                         │
├─────────────────────────────────────────┤
│  LangChain Core (核心抽象)              │
└─────────────────────────────────────────┘
```

### 1.2 第一個 LLM 呼叫

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage

# 初始化 ChatModel
llm = ChatOpenAI(
    model="gpt-4",
    temperature=0.7,  # 控制創意度 (0-1)
    max_tokens=500    # 最大輸出長度
)

# 簡單呼叫
response = llm.invoke([HumanMessage(content="你好,請簡短介紹國泰人壽")])

print("🤖 AI 回應:")
print(response.content)

### 1.3 LLM vs ChatModel 差異

| 特性 | LLM | ChatModel |
|------|-----|----------|
| 輸入格式 | 純文字字串 | Message 物件列表 |
| 輸出格式 | 純文字字串 | AIMessage 物件 |
| 支援角色 | 無 | System/Human/AI |
| 對話歷史 | 需手動管理 | 原生支援 |
| 範例模型 | text-davinci-003 | gpt-4, gpt-3.5-turbo |

In [ ]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

# 使用 ChatModel 的多輪對話
messages = [
    SystemMessage(content="你是國泰人壽的專業客服人員,請用友善且專業的態度回答客戶問題。"),
    HumanMessage(content="請問你們有哪些壽險商品?"),
    AIMessage(content="國泰人壽提供多元壽險商品,包括終身壽險、定期壽險、投資型保單等。"),
    HumanMessage(content="終身壽險的保障範圍是什麼?")
]

response = llm.invoke(messages)
print("💼 客服回應:")
print(response.content)

## Part 2: PromptTemplate 提示詞模板

### 2.1 為什麼需要 PromptTemplate?

❌ **不使用模板**:
```python
prompt = f"請查詢保單號碼 {policy_id} 的資訊"
```
問題: 難以維護、容易出錯、無法重複使用

✅ **使用模板**:
```python
template = PromptTemplate(
    template="請查詢保單號碼 {policy_id} 的資訊",
    input_variables=["policy_id"]
)
```
優點: 可重複使用、參數化、易於測試

### 2.2 基礎 PromptTemplate

In [ ]:
from langchain.prompts import PromptTemplate

# 建立保單查詢模板
policy_template = PromptTemplate(
    input_variables=["policy_id", "customer_name"],
    template="""
客戶姓名: {customer_name}
保單號碼: {policy_id}

請查詢該保單的以下資訊:
1. 保單狀態
2. 保障內容
3. 繳費狀況
4. 受益人資訊

請以專業且易懂的方式說明。
"""
)

# 使用模板
prompt = policy_template.format(
    policy_id="CL202401001",
    customer_name="王小明"
)

print("📄 生成的提示詞:")
print(prompt)

### 2.3 ChatPromptTemplate (對話模板)

In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 建立對話模板
system_template = "你是國泰人壽的{role},擁有{years}年的從業經驗。請用專業且親切的態度回答問題。"
human_template = "{customer_question}"

chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_template),
    HumanMessagePromptTemplate.from_template(human_template)
])

# 格式化訊息
messages = chat_prompt.format_messages(
    role="資深理財顧問",
    years=10,
    customer_question="請問投資型保單適合我這種保守型投資人嗎?"
)

print("💬 對話訊息:")
for msg in messages:
    print(f"{msg.__class__.__name__}: {msg.content}")

## Part 3: Chain 鏈接組件

### 3.1 什麼是 Chain?

Chain 是 LangChain 的核心概念,用於將多個組件串接起來:

```
Input → PromptTemplate → LLM → OutputParser → Output
```

### 3.2 LLMChain 基礎用法

In [ ]:
from langchain.chains import LLMChain

# 建立 Chain
llm = ChatOpenAI(model="gpt-4", temperature=0.7)

prompt = PromptTemplate(
    input_variables=["product_type"],
    template="請用 3 個重點說明國泰人壽的{product_type}有哪些特色和優勢。"
)

chain = LLMChain(llm=llm, prompt=prompt)

# 執行 Chain
result = chain.run(product_type="醫療險")
print("🏥 醫療險特色:")
print(result)

### 3.3 使用 LCEL (LangChain Expression Language)

In [ ]:
# LCEL 是更現代的 Chain 寫法,使用 | 運算子
from langchain_core.output_parsers import StrOutputParser

# 建立 Chain: Prompt → LLM → Parser
chain = prompt | llm | StrOutputParser()

# 執行
result = chain.invoke({"product_type": "儲蓄險"})
print("💰 儲蓄險特色:")
print(result)

### 3.4 SequentialChain 順序鏈

In [ ]:
from langchain.chains import SimpleSequentialChain

# Chain 1: 分析客戶需求
analyze_prompt = PromptTemplate(
    input_variables=["customer_info"],
    template="根據以下客戶資訊,分析其保險需求:\n{customer_info}\n\n分析結果:"
)
analyze_chain = LLMChain(llm=llm, prompt=analyze_prompt)

# Chain 2: 推薦產品
recommend_prompt = PromptTemplate(
    input_variables=["needs_analysis"],
    template="根據需求分析:\n{needs_analysis}\n\n推薦適合的國泰人壽保險商品:"
)
recommend_chain = LLMChain(llm=llm, prompt=recommend_prompt)

# 組合成順序鏈
overall_chain = SimpleSequentialChain(
    chains=[analyze_chain, recommend_chain],
    verbose=True  # 顯示中間過程
)

# 執行
customer_data = """
姓名: 陳小華
年齡: 35 歲
職業: 工程師
家庭狀況: 已婚,有 2 個小孩 (5 歲、3 歲)
年收入: 150 萬
現有保障: 僅有公司團保
"""

result = overall_chain.run(customer_data)
print("\n📊 最終推薦:")
print(result)

## Part 4: Output Parsers 輸出解析

### 4.1 為什麼需要 Output Parser?

LLM 的輸出通常是純文字,但我們常需要結構化資料 (JSON、List 等)

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser

# 建立列表解析器
parser = CommaSeparatedListOutputParser()

# 取得格式指示
format_instructions = parser.get_format_instructions()

# 建立提示詞
prompt = PromptTemplate(
    template="列出國泰人壽最受歡迎的 5 個保險商品。\n{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": format_instructions}
)

# 建立 Chain
chain = prompt | llm | parser

# 執行
result = chain.invoke({})
print("📋 熱門商品列表:")
print(f"類型: {type(result)}")
for i, product in enumerate(result, 1):
    print(f"{i}. {product}")

### 4.2 Pydantic Output Parser (結構化輸出)

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# 定義保單資料結構
class PolicyInfo(BaseModel):
    policy_id: str = Field(description="保單號碼")
    policy_type: str = Field(description="保單類型")
    status: str = Field(description="保單狀態")
    premium: int = Field(description="年繳保費")
    coverage: List[str] = Field(description="保障項目列表")

# 建立解析器
parser = PydanticOutputParser(pydantic_object=PolicyInfo)

# 建立提示詞
prompt = PromptTemplate(
    template="請生成一個範例保單資料。\n{format_instructions}",
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# 執行
chain = prompt | llm | parser
result = chain.invoke({})

print("📄 保單資料:")
print(f"類型: {type(result)}")
print(f"保單號碼: {result.policy_id}")
print(f"保單類型: {result.policy_type}")
print(f"保單狀態: {result.status}")
print(f"年繳保費: ${result.premium:,}")
print(f"保障項目: {', '.join(result.coverage)}")

## Part 5: 實戰案例 - 國泰人壽保單查詢助理

### 整合所有學到的技術

In [ ]:
from datetime import datetime

class PolicyQueryResult(BaseModel):
    """保單查詢結果"""
    policy_id: str = Field(description="保單號碼")
    customer_name: str = Field(description="客戶姓名")
    status: str = Field(description="保單狀態:有效/失效/等待期")
    next_payment_date: str = Field(description="下次繳費日期 (YYYY-MM-DD)")
    summary: str = Field(description="保單摘要說明")
    suggestions: List[str] = Field(description="客服建議列表")

# 建立解析器
query_parser = PydanticOutputParser(pydantic_object=PolicyQueryResult)

# 建立系統提示詞
system_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="""
你是國泰人壽的智能客服系統。
當客戶查詢保單時,你需要:
1. 確認客戶身分和保單號碼
2. 提供保單狀態資訊
3. 說明下次繳費日期
4. 提供專業建議

請務必保持專業、準確、親切的態度。
"""),
    HumanMessagePromptTemplate.from_template("""
客戶查詢資訊:
保單號碼: {policy_id}
客戶姓名: {customer_name}
查詢時間: {query_time}

{format_instructions}
""")
])

# 建立查詢 Chain
policy_query_chain = (
    system_prompt 
    | llm 
    | query_parser
)

# 執行查詢
result = policy_query_chain.invoke({
    "policy_id": "CL202401001",
    "customer_name": "王小明",
    "query_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "format_instructions": query_parser.get_format_instructions()
})

# 顯示結果
print("="*60)
print("🏢 國泰人壽保單查詢系統")
print("="*60)
print(f"\n📋 保單號碼: {result.policy_id}")
print(f"👤 客戶姓名: {result.customer_name}")
print(f"✅ 保單狀態: {result.status}")
print(f"📅 下次繳費日: {result.next_payment_date}")
print(f"\n📝 保單摘要:\n{result.summary}")
print(f"\n💡 專業建議:")
for i, suggestion in enumerate(result.suggestions, 1):
    print(f"   {i}. {suggestion}")
print("\n" + "="*60)

## 🎯 練習題

### 練習 1: 建立理賠查詢系統

**需求**:
1. 定義 ClaimInfo 資料結構 (理賠編號、狀態、金額、處理進度)
2. 建立 PromptTemplate
3. 使用 Pydantic Parser
4. 建立完整的查詢 Chain

In [ ]:
# TODO: 在此實作理賠查詢系統

class ClaimInfo(BaseModel):
    # TODO: 定義欄位
    pass

# TODO: 建立 Parser 和 Chain

### 練習 2: 多步驟產品推薦

**需求**:
1. Step 1: 分析客戶年齡、收入、家庭狀況
2. Step 2: 評估風險承受度
3. Step 3: 推薦 3 個適合的保險商品
4. 使用 SequentialChain 串接

In [ ]:
# TODO: 建立多步驟推薦系統

# Step 1: 客戶分析
# Step 2: 風險評估  
# Step 3: 產品推薦

### 練習 3: 保費試算工具

**需求**:
1. 輸入: 年齡、性別、保額、保障年期
2. 輸出結構化的試算結果 (年繳/月繳/總保費)
3. 提供繳費建議

In [ ]:
# TODO: 建立保費試算工具

class PremiumCalculation(BaseModel):
    # TODO: 定義試算結果結構
    pass

## ✅ 練習解答

### 練習 1 解答: 理賠查詢系統

In [ ]:
class ClaimInfoSolution(BaseModel):
    """理賠資訊"""
    claim_id: str = Field(description="理賠編號")
    policy_id: str = Field(description="保單號碼")
    claim_type: str = Field(description="理賠類型:醫療/意外/身故")
    claim_amount: int = Field(description="理賠金額")
    status: str = Field(description="處理狀態:審核中/已核准/已撥款/已拒絕")
    progress_percentage: int = Field(description="處理進度百分比 0-100")
    estimated_completion: str = Field(description="預計完成日期")
    required_documents: List[str] = Field(description="需補件列表")

# 建立解析器
claim_parser = PydanticOutputParser(pydantic_object=ClaimInfoSolution)

# 建立提示詞
claim_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="你是國泰人壽理賠部門的客服系統,請提供準確的理賠進度資訊。"),
    HumanMessagePromptTemplate.from_template("""
查詢理賠案件:
理賠編號: {claim_id}
客戶姓名: {customer_name}

{format_instructions}
""")
])

# 建立 Chain
claim_chain = claim_prompt | llm | claim_parser

# 執行查詢
claim_result = claim_chain.invoke({
    "claim_id": "CLM20240315001",
    "customer_name": "李小華",
    "format_instructions": claim_parser.get_format_instructions()
})

print("\n🏥 理賠查詢結果")
print("="*50)
print(f"理賠編號: {claim_result.claim_id}")
print(f"保單號碼: {claim_result.policy_id}")
print(f"理賠類型: {claim_result.claim_type}")
print(f"理賠金額: ${claim_result.claim_amount:,}")
print(f"處理狀態: {claim_result.status}")
print(f"處理進度: {claim_result.progress_percentage}%")
print(f"預計完成: {claim_result.estimated_completion}")
if claim_result.required_documents:
    print(f"\n📎 需補件:")
    for doc in claim_result.required_documents:
        print(f"  - {doc}")
else:
    print("\n✅ 無需補件")

### 練習 2 解答: 多步驟產品推薦

In [ ]:
# Step 1: 客戶分析 Chain
analysis_prompt = PromptTemplate(
    input_variables=["customer_profile"],
    template="""
分析以下客戶資料,評估其生涯階段和保險需求:

{customer_profile}

請分析:
1. 生涯階段
2. 主要風險缺口
3. 保障優先順序
"""
)
analysis_chain = analysis_prompt | llm | StrOutputParser()

# Step 2: 風險評估 Chain
risk_prompt = PromptTemplate(
    input_variables=["analysis"],
    template="""
根據客戶分析結果:
{analysis}

評估客戶的風險承受度 (保守/穩健/積極),並說明理由。
"""
)
risk_chain = risk_prompt | llm | StrOutputParser()

# Step 3: 產品推薦 Chain
recommend_prompt = PromptTemplate(
    input_variables=["risk_assessment"],
    template="""
基於風險評估:
{risk_assessment}

推薦 3 個國泰人壽適合的保險商品,並說明推薦理由。
"""
)
recommend_chain = recommend_prompt | llm | StrOutputParser()

# 組合成順序鏈 (使用 LCEL)
from langchain_core.runnables import RunnablePassthrough

full_chain = (
    {"customer_profile": RunnablePassthrough()}
    | analysis_chain
    | {"analysis": RunnablePassthrough()}
    | risk_chain
    | {"risk_assessment": RunnablePassthrough()}
    | recommend_chain
)

# 測試
customer = """
姓名: 張美玲
年齡: 28 歲
職業: 軟體工程師
婚姻: 未婚
年收入: 120 萬
存款: 80 萬
投資經驗: 有購買基金經驗
現有保障: 公司團保 + 意外險
"""

result = full_chain.invoke(customer)
print("\n💼 產品推薦結果")
print("="*60)
print(result)

### 練習 3 解答: 保費試算工具

In [ ]:
class PremiumCalculationSolution(BaseModel):
    """保費試算結果"""
    product_name: str = Field(description="商品名稱")
    coverage_amount: int = Field(description="保額")
    coverage_years: int = Field(description="保障年期")
    annual_premium: int = Field(description="年繳保費")
    monthly_premium: int = Field(description="月繳保費")
    total_premium: int = Field(description="總繳保費")
    payment_suggestion: str = Field(description="繳費建議")
    benefit_summary: List[str] = Field(description="保障摘要")

# 建立解析器
premium_parser = PydanticOutputParser(pydantic_object=PremiumCalculationSolution)

# 建立提示詞
premium_prompt = PromptTemplate(
    input_variables=["age", "gender", "coverage", "years"],
    template="""
請為以下條件試算國泰人壽終身壽險保費:

年齡: {age} 歲
性別: {gender}
保額: ${coverage:,}
保障年期: {years} 年

請提供完整的保費試算結果和繳費建議。

{format_instructions}
""",
    partial_variables={"format_instructions": premium_parser.get_format_instructions()}
)

# 建立 Chain
premium_chain = premium_prompt | llm | premium_parser

# 執行試算
calc_result = premium_chain.invoke({
    "age": 30,
    "gender": "男",
    "coverage": 2000000,
    "years": 20
})

print("\n💰 保費試算結果")
print("="*60)
print(f"商品: {calc_result.product_name}")
print(f"保額: ${calc_result.coverage_amount:,}")
print(f"保障年期: {calc_result.coverage_years} 年")
print(f"\n💵 保費資訊:")
print(f"  年繳: ${calc_result.annual_premium:,}")
print(f"  月繳: ${calc_result.monthly_premium:,}")
print(f"  總繳: ${calc_result.total_premium:,}")
print(f"\n💡 繳費建議:\n{calc_result.payment_suggestion}")
print(f"\n✅ 保障內容:")
for benefit in calc_result.benefit_summary:
    print(f"  • {benefit}")

## 📝 學習檢查清單

完成本 Notebook 後,你應該能夠:

- [ ] 理解 LangChain 的核心架構
- [ ] 區分 LLM 和 ChatModel 的差異
- [ ] 使用 PromptTemplate 建立可重複使用的提示詞
- [ ] 使用 ChatPromptTemplate 建立對話模板
- [ ] 理解 Chain 的概念並建立簡單的 LLMChain
- [ ] 使用 LCEL (LangChain Expression Language)
- [ ] 使用 SequentialChain 建立多步驟流程
- [ ] 使用 Output Parsers 取得結構化輸出
- [ ] 使用 Pydantic 定義複雜的輸出結構
- [ ] 整合所有技術建立實用的應用

## 🎓 重點回顧

### LangChain 核心概念
1. **Components**: Prompts, Models, Parsers, Memory, Tools
2. **Chains**: 串接多個組件形成完整流程
3. **LCEL**: 使用 `|` 運算子的現代化 Chain 語法

### 最佳實踐
1. 使用 PromptTemplate 管理提示詞
2. 使用 Pydantic 定義結構化輸出
3. 使用 ChatModel 而非 LLM (支援角色和對話歷史)
4. 使用 LCEL 建立可讀性高的 Chain

### 下一步
繼續前往 `02-langchain-agents.ipynb` 學習如何建立 LangChain Agent 👉